In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score, matthews_corrcoef, precision_score, recall_score
from sklearn.feature_selection import SequentialFeatureSelector, VarianceThreshold, RFE
from sklearn.svm import SVR, SVC

In [78]:
feat_names = []
for i in range(334):
    feat_names.append('feat' + str(i + 1))

df_train = pd.read_csv('train10000.csv', names=feat_names)
df_train_targets = pd.read_csv('train10000_Label.csv', names=['targets'])

df_test = pd.read_csv('test10000.csv', names=feat_names)
df_test_targets = pd.read_csv('test10000_label.csv', names=['targets'])

In [79]:
df_train=(df_train-df_train.min())/(df_train.max()-df_train.min())
df_test=(df_test-df_test.min())/(df_test.max()-df_test.min())

In [80]:
df_train = df_train.join(df_train_targets['targets'])
df_train_default = df_train
df_train.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,1.000000,0.345679,0.000001,0.345679,0.666667,0.000001,1.000000,0.000003,0.000002,1.000000,...,0.208791,1.000000,0.428571,0.150,0.275862,0.120879,0.355263,0.153846,0.069620,1
1,0.000001,0.395062,0.000000,0.395062,0.666667,0.000000,0.000003,0.000006,0.000004,0.000000,...,0.263736,0.898990,0.265306,0.275,0.068966,0.109890,0.394737,0.102362,0.091139,1
2,1.000000,0.395062,0.000000,0.395062,0.666667,0.000000,0.000003,1.000000,0.000006,1.000000,...,0.219780,0.989899,0.346939,0.175,0.068966,0.109890,0.315789,0.204119,0.066456,1
3,0.000001,0.493827,0.000000,0.493827,0.555556,0.000001,0.000003,1.000000,0.000007,0.000001,...,0.208791,0.838384,0.244898,0.200,0.137931,0.043956,0.407895,0.169594,0.104430,1
4,1.000000,0.567901,0.000000,0.567901,0.666667,0.000000,0.000003,0.000001,0.000001,1.000000,...,0.252747,1.000000,0.448980,0.100,0.275862,0.483516,0.355263,0.086614,0.007595,1


In [81]:
df_test = df_test.join(df_test_targets['targets'])
df_test.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,0.000001,0.222222,0.000002,0.222222,0.788013,0.000000,1.000000,1.0,0.000006,0.000001,...,0.098901,0.979798,0.705882,0.075,0.20,0.070707,0.148936,0.210176,0.033373,0
1,0.000000,0.222222,0.000000,0.246914,0.655938,0.000000,0.000003,1.0,0.000006,0.000000,...,0.076923,0.818182,0.558824,0.050,0.15,0.030303,0.053191,0.160509,0.061936,0
2,1.000000,0.345679,0.000000,0.345679,0.655938,0.000000,0.000003,1.0,0.000006,1.000000,...,0.109890,0.898990,0.647059,0.100,0.15,0.121212,0.138298,0.253180,0.042995,0
3,0.000001,0.296296,0.000000,0.296296,0.733629,0.000000,0.000003,1.0,1.000000,0.000000,...,0.142857,0.848485,0.176471,0.100,0.55,0.020202,0.276596,0.344034,0.082682,0
4,1.000000,0.123457,0.000000,0.123457,0.877913,0.000001,1.000000,1.0,0.000006,1.000000,...,0.131868,0.888889,0.529412,0.100,0.25,0.212121,0.202128,0.198667,0.068250,0


In [82]:
df_train.shape, df_test.shape

((10000, 335), (10000, 335))

In [83]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
df_train.isnull().sum()

feat1      0
feat2      0
feat3      0
feat4      0
feat5      0
          ..
feat331    0
feat332    0
feat333    0
feat334    0
targets    0
Length: 335, dtype: int64

In [84]:
y_train = df_train['targets'].values
df_train = df_train.drop(columns=['targets'])

y_test = df_test['targets'].values
df_test = df_test.drop(columns=['targets'])

In [110]:
#------------feature selection-----------------

In [85]:
selector = VarianceThreshold(threshold=0.01)
print(df_train.shape)
#df_train = pd.DataFrame(df_train_array)
selector.fit_transform(df_train)
feat_mask = selector.get_support()
#print(feat_mask)
#print(df_train.shape)

count = 0
for i in range(len(feat_mask)):
    #print(feat_mask[i])
    if feat_mask[i] == False:
        df_train = df_train.drop(df_train.columns[i - count], axis=1)
        df_test = df_test.drop(df_test.columns[i - count], axis=1)
        count = count + 1

print(df_train.shape, df_test.shape)

# estimator = SVR(kernel="linear")
# selector = RFE(estimator, n_features_to_select=160, step=1)
# selector = selector.fit(df_train, y_train)
# print(selector.support_)

(10000, 334)
(10000, 165) (10000, 165)


In [86]:
df_train_featselect = df_train.join(pd.DataFrame(y_train, columns=['targets']))

In [87]:
df_train.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat312,feat314,feat317,feat323,feat324,feat325,feat327,feat328,feat331,feat332
0,1.000000,0.345679,0.000001,0.345679,0.666667,0.000001,1.000000,0.000003,0.000002,1.000000,...,0.636364,0.000000,0.000000,0.070707,1.000000,0.7375,1.000000,0.428571,0.120879,0.355263
1,0.000001,0.395062,0.000000,0.395062,0.666667,0.000000,0.000003,0.000006,0.000004,0.000000,...,0.363636,0.119048,0.068493,0.101010,0.888889,0.6375,0.898990,0.265306,0.109890,0.394737
2,1.000000,0.395062,0.000000,0.395062,0.666667,0.000000,0.000003,1.000000,0.000006,1.000000,...,0.595960,0.000000,0.000000,0.020202,0.989899,0.7625,0.989899,0.346939,0.109890,0.315789
3,0.000001,0.493827,0.000000,0.493827,0.555556,0.000001,0.000003,1.000000,0.000007,0.000001,...,0.404040,0.023810,0.013699,0.070707,0.818182,0.6250,0.838384,0.244898,0.043956,0.407895
4,1.000000,0.567901,0.000000,0.567901,0.666667,0.000000,0.000003,0.000001,0.000001,1.000000,...,0.676768,0.000000,0.000000,0.000000,1.000000,0.7125,1.000000,0.448980,0.483516,0.355263


In [88]:
df_train = df_train.join(pd.DataFrame(y_train, columns=['targets']))

In [89]:
#------------imbalanced data handeling------------------

In [90]:
targetGrouping = df_train.groupby(df_train['targets'], group_keys=False)
stratified = pd.DataFrame(targetGrouping.apply(lambda x: x.sample(targetGrouping.size().min()))).reset_index(drop=True)
stratified['targets'].value_counts()

df_train = stratified
df_train.shape

(1818, 166)

In [91]:
#------------SVM---------------

In [101]:
roc_cv_svm = []
mcc_cv_svm = []
precision_cv_svm = []
recall_cv_svm = []
bestVal = 100.0
print('Results for 10-fold-CV of SVM:\n---------------------------------')
for i in range(10):
    print(f'Run-{i + 1}:\n-----------')
    train, _ = train_test_split(df_train, test_size=0.1, random_state=i*2, shuffle=True)
    
    #---------Normalizing-------------
    scaler = StandardScaler()
    y_train = train['targets'].values
    X_train = train.drop(columns=['targets'])
    X_train = scaler.fit_transform(X_train)
    
    clf = SVC(kernel='rbf', gamma=0.001)
    clf.fit(X_train, y_train)
    
    pred = clf.predict(X_train)
    roc_area = roc_auc_score(y_train, pred)
    mcc = matthews_corrcoef(y_train, pred)
    precision = precision_score(y_train, pred)
    recall = recall_score(y_train, pred)
    
    if roc_area < bestVal:
        bestVal = roc_area
        X_test = scaler.transform(df_test)
        bestModel = clf.predict(X_test)
    
    print(f'ROC area: {roc_area}')
    print(f'MCC: {mcc}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}\n')
    
    roc_cv_svm.append(roc_area)
    mcc_cv_svm.append(mcc)
    precision_cv_svm.append(precision)
    recall_cv_svm.append(recall)

print(f'\n\nCV ROC area: {np.mean(roc_cv_svm)}')
print(f'CV MCC: {np.mean(mcc_cv_svm)}')
print(f'CV Precision: {np.mean(precision_cv_svm)}')
print(f'CV Recall: {np.mean(recall_cv_svm)}')

Results for 10-fold-CV of SVM:
---------------------------------
Run-1:
-----------
ROC area: 0.6368589930401436
MCC: 0.2750696819539947
Precision: 0.6512890094979648
Recall: 0.587515299877601

Run-2:
-----------
ROC area: 0.6433517726065439
MCC: 0.287120984181208
Precision: 0.6571798188874515
Recall: 0.6142684401451027

Run-3:
-----------
ROC area: 0.6419536107125777
MCC: 0.2858819582135915
Precision: 0.6620498614958449
Recall: 0.5829268292682926

Run-4:
-----------
ROC area: 0.637490416560184
MCC: 0.2755817672303423
Precision: 0.6465968586387435
Recall: 0.6046511627906976

Run-5:
-----------
ROC area: 0.6342693337797121
MCC: 0.27013511528563533
Precision: 0.654320987654321
Recall: 0.5788834951456311

Run-6:
-----------
ROC area: 0.631579655302283
MCC: 0.26371027773987826
Precision: 0.6431372549019608
Recall: 0.5985401459854015

Run-7:
-----------
ROC area: 0.6319501330383569
MCC: 0.2639002660767138
Precision: 0.6373493975903615
Recall: 0.6373493975903615

Run-8:
-----------
ROC area:

In [108]:
write_prop=open('predict.csv', 'w')

for i in range(len(bestModel)):
    write_prop.write(str(bestModel[i]) + '\n')

write_prop.close()
print('predictions saved in predict.csv .......')

predictions saved in predict.csv .......


In [109]:
print('Resullts on best model:\n------------------------')
print(f'Precision: {precision_score(y_test, bestModel)}')
print(f'Recall: {recall_score(y_test, bestModel)}')
print(f'MCC: {matthews_corrcoef(y_test, bestModel)}')
print(f'ROC Area: {roc_auc_score(y_test, bestModel)}')

Resullts on best model:
------------------------
Precision: 0.14026284348864995
Recall: 0.625133120340788
MCC: 0.13484103401495018
ROC Area: 0.6140233530188655
